### 加载数据集

In [2]:
from utils import load_corpus, stopwords

TRAIN_PATH = "./data/weibo_dataset/train.txt"
TEST_PATH = "./data/weibo_dataset/test.txt"

In [3]:
# 分别加载训练集和测试集
train_data = load_corpus(TRAIN_PATH)
test_data = load_corpus(TEST_PATH)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\63156\AppData\Local\Temp\jieba.cache
Loading model cost 0.518 seconds.
Prefix dict has been built successfully.


In [4]:
import pandas as pd

df_train = pd.DataFrame(train_data, columns=["words", "label"])
df_test = pd.DataFrame(test_data, columns=["words", "label"])
df_train.head()

,words,label
0,书中 自有 黄金屋 书中 自有 颜如玉 沿着 岁月 的 长河 跋涉 或是 风光旖旎 或是 姹...,1
1,这是 英超 被 黑 的 最惨 的 一次 二哈 二哈 十几年来 中国 只有 孙继海 董方卓 郑...,0
2,中国 远洋 海运 集团 副总经理 俞曾 港 月 日 在 上 表示 中央 企业 走 出去 是 ...,1
3,看 流星花园 其实 也 还好 啦 现在 的 观念 以及 时尚 眼光 都 不一样 了 或许 十...,1
4,汉武帝 的 罪己 诏 的 真实性 尽管 存在 着 争议 然而 轮台 罪己 诏 作为 中国 历...,1


### 特征编码（Tf-Idf模型）

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(token_pattern='\[?\w+\]?', 
                             stop_words=stopwords)
X_train = vectorizer.fit_transform(df_train["words"])
y_train = df_train["label"]

d:\Python3.10.0\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['元', '吨', '数', '末'] not in stop_words.
  warnings.warn(


In [6]:
X_test = vectorizer.transform(df_test["words"])
y_test = df_test["label"]

### 训练模型&测试

In [7]:
from sklearn import svm

clf = svm.SVC()
clf.fit(X_train, y_train)

SVC()

In [8]:
# 在测试集上用模型预测结果
y_pred = clf.predict(X_test)

In [9]:
# 测试集效果检验
from sklearn import metrics

print(metrics.classification_report(y_test, y_pred, digits=3))
print("准确率: {:.3f}".format(metrics.accuracy_score(y_test, y_pred)))

auc_score = metrics.roc_auc_score(y_test, y_pred)
print("AUC:", auc_score)

              precision    recall  f1-score   support

           0      0.817     0.690     0.748       155
           1      0.870     0.930     0.899       345

    accuracy                          0.856       500
   macro avg      0.843     0.810     0.824       500
weighted avg      0.853     0.856     0.852       500

准确率: 0.856
AUC: 0.8103786816269284


### 手动输入句子，判断情感倾向

In [10]:
from utils import processing

strs = ["只要流过的汗与泪都能化作往后的明亮，就值得你为自己喝彩", "烦死了！为什么周末还要加班[愤怒]"]
words = [processing(s) for s in strs]
vec = vectorizer.transform(words)

In [11]:
output = clf.predict(vec)
output

array([1, 0], dtype=int64)